In [56]:
import sys
import os
import ujson
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from sklearn.linear_model import LinearRegression
from epivizfileserver.measurements import WebServerMeasurement
sys.path.insert(0,'C:/Users/Kyle/Documents/Python Scripts/Envs/research_epiviz/efeedcomputation/src/epivizfeedcompute')
from stats import BaseStats, TtestBlock, TtestExp, Correlation, CorrelationGeneSignal, OverlapBlock
import pickle


In [57]:
data = []
mid = "lung_cancer"
start=1
end=166151839
signal_data = pd.read_csv('timp2014_probelevel_beta_202001091200.csv')[["chr", "start", "end", "lung_normal", "lung_cancer"]]
signal_data = signal_data.where((signal_data.start >= start) & (signal_data.end <= end))
signal_data = signal_data.where((signal_data.chr != 'chrY'))
signal_data = signal_data.dropna()

In [58]:
signal_data

,chr,start,end,lung_normal,lung_cancer
0,chr1,15865.0,15865.0,0.9026,0.8800
1,chr1,18827.0,18827.0,0.6465,0.6145
2,chr1,29407.0,29407.0,0.0731,0.0985
3,chr1,29425.0,29425.0,0.1858,0.2073
4,chr1,29435.0,29435.0,0.0391,0.0353
...,...,...,...,...,...
485091,chrX,154842736.0,154842736.0,0.1320,0.2107
485092,chrX,154842841.0,154842841.0,0.3342,0.3300
485093,chrX,154842856.0,154842856.0,0.5510,0.4668
485094,chrX,154843228.0,154843228.0,0.6587,0.6618


In [59]:
cpg_islands = pd.read_csv('cpg_islands_202001141145.csv')
cpg_islands = cpg_islands.where((cpg_islands.chr != 'chrY')).dropna()
cpg_islands

,id,chr,start,end
0,1.0,chr1,28735.0,29810.0
1,2.0,chr1,135124.0,135563.0
2,3.0,chr1,327790.0,328229.0
3,4.0,chr1,437151.0,438164.0
4,5.0,chr1,449273.0,450544.0
...,...,...,...,...
27532,27533.0,chrX,154687387.0,154688519.0
27533,27534.0,chrX,154689346.0,154689788.0
27534,27535.0,chrX,154842112.0,154842719.0
27535,27536.0,chrX,155110788.0,155111177.0


In [60]:
# config_file = os.getcwd() + "/config.json"
# config_measurements = []
# with open(config_file, "r") as config_file:
#     data = ujson.loads(config_file.read())
#     # computations = data["computations"]
#     # info = data["dataSources"]
#     # pval_threshold = data["pval_threshold"]
#     if data["measurements"] is not None:
#         measurements = data["measurements"]
#         for m in measurements: 
#             tm = WebServerMeasurement(m['type'], m['id'], m['name'],  "http://54.157.53.251/api/", 
#                     m['datasourceId'], m['datasourceGroup'], m['annotation'], m['metadata'])
#             if tm.annotation is None:
#                 tm.annotation = {}
#             tm.annotation["datatype"] = m["datatype"]
#             config_measurements.append(tm)
# #ESR1 chr6: 150204511 - 157531913
# #ATOH7 chr10: 63661013 - 71027315
# chrom = "chr11"
# start=1
# end=166151839
# # filter for overlap measuremnets
# overlap_measurements = [m for m in config_measurements if m.mid in ["cpg_islands", "promoter"] ]
# overlap_measurements
# data = []

# for m in overlap_measurements:
#     data.append(m.get_data(chrom, start, end))
# print("done")

In [78]:
def split_n_buckets(dataframe, n):

    means = []
    dfs = np.array_split(dataframe, n,axis = 0) 
#     print(dfs)
    for i in range(n):
        means.append(np.mean(dfs[i]))
    return means

In [62]:
dataF = pd.DataFrame() 
for i in range(1,23):
    query = 'http://54.157.53.251/api/?requestId=11&version=5&action=getValues&datasourceGroup=umd&datasource=gene_expression_barcode_subtype&measurement=lung___tumor,lung___normal&seqName=chr{}&start=1&end=166151839&genome=&_=1573233064747'.format(i)
    result = requests.get(query)
    # res = umsgpack.unpackb(result.content)
    res = result.json()

    data = res['data']

    if data['rows']['useOffset']:
        data['rows']['values']['start'] = np.cumsum(data['rows']['values']['start'])
        data['rows']['values']['end'] = np.cumsum(data['rows']['values']['end'])

    # convert json to dataframe
    records = {}

    for key in data['rows']['values'].keys():
        if key not in ["id", "strand", "metadata"]:
            records[key] = data['rows']['values'][key]

    for key in data['rows']['values']['metadata'].keys():
        records[key] = data['rows']['values']['metadata'][key]

    for key in data['values']['values'].keys():
        records[key] = data['values']['values'][key]
    temp = pd.DataFrame(records)
    dataF = pd.concat([dataF,temp])

dataF.head()


,end,start,chr,lung___normal,lung___tumor
0,894670.0,861118.0,chr1,0.0,0.025194
1,901095.0,895967.0,chr1,0.0,0.007752
2,901095.0,895967.0,chr1,0.0,0.003876
3,910488.0,901877.0,chr1,0.0,0.027132
4,917497.0,910579.0,chr1,0.0,0.007752


In [63]:
for i in ['X']:
    query = 'http://54.157.53.251/api/?requestId=11&version=5&action=getValues&datasourceGroup=umd&datasource=gene_expression_barcode_subtype&measurement=lung___tumor,lung___normal&seqName=chr{}&start=1&end=166151839&genome=&_=1573233064747'.format(i)
    result = requests.get(query)
    # res = umsgpack.unpackb(result.content)
    res = result.json()

    data = res['data']

    if data['rows']['useOffset']:
        data['rows']['values']['start'] = np.cumsum(data['rows']['values']['start'])
        data['rows']['values']['end'] = np.cumsum(data['rows']['values']['end'])

    # convert json to dataframe
    records = {}

    for key in data['rows']['values'].keys():
        if key not in ["id", "strand", "metadata"]:
            records[key] = data['rows']['values'][key]

    for key in data['rows']['values']['metadata'].keys():
        records[key] = data['rows']['values']['metadata'][key]

    for key in data['values']['values'].keys():
        records[key] = data['values']['values'][key]
    temp = pd.DataFrame(records)
    dataF = pd.concat([dataF,temp])


In [64]:
dataF.shape
# dataF['chr'].unique()

(28589, 5)

In [65]:
cpg_islands

,id,chr,start,end
0,1.0,chr1,28735.0,29810.0
1,2.0,chr1,135124.0,135563.0
2,3.0,chr1,327790.0,328229.0
3,4.0,chr1,437151.0,438164.0
4,5.0,chr1,449273.0,450544.0
...,...,...,...,...
27532,27533.0,chrX,154687387.0,154688519.0
27533,27534.0,chrX,154689346.0,154689788.0
27534,27535.0,chrX,154842112.0,154842719.0
27535,27536.0,chrX,155110788.0,155111177.0


In [79]:
# run this version
in_gene = []
methy_upstream = []
methy_downstream = []
upstream = 1000
downstream = 3000
temp = dataF.head(100)

for index, row in temp.iterrows():
    in_gene_buckets = split_n_buckets(signal_data.where((((row.start <= signal_data.start) & (signal_data.start <= row.end)) | ((row.start <= signal_data.end) & (signal_data.end <= row.end))) & (row.chr == signal_data.chr)).dropna()[mid].tolist(), 20)
    methy_upstream_buckets = split_n_buckets(signal_data.where((((row.start - upstream<= signal_data.start) & (signal_data.start <= row.end)) | ((row.start - upstream <= signal_data.end) & (signal_data.end <= row.end))) & (row.chr == signal_data.chr)).dropna()[mid].tolist(), 10)
    methy_downstream_buckets = split_n_buckets(signal_data.where((((row.start <= signal_data.start) & (signal_data.start <= row.end + downstream)) | ((row.start <= signal_data.end) & (signal_data.end <= row.end + downstream))) & (row.chr == signal_data.chr)).dropna()[mid].tolist(), 10)
    in_gene.append(in_gene_buckets)
    methy_upstream.append(methy_upstream_buckets)
    methy_downstream.append(methy_downstream_buckets)


In [75]:
in_gene_buckets

[0.18645,
 0.678575,
 0.624225,
 0.333175,
 0.31015,
 0.5879749999999999,
 0.6307999999999999,
 0.09246666666666666,
 0.11763333333333333,
 0.0453,
 0.3376666666666666,
 0.6946666666666665,
 0.7041,
 0.8550666666666666,
 0.8879333333333334,
 0.8321666666666667,
 0.7465333333333333,
 0.8254,
 0.04623333333333333,
 0.0477]

In [30]:
in_gene_islands = []
for index, row in dataF.iterrows():
    # gets number of rows
    cpg_island_count = cpg_islands.where((row.start <= cpg_islands.start) & (row.end >= cpg_islands.end) & (row.chr == cpg_islands.chr)).dropna().shape[0]
    contains_cpg_island = False
    if cpg_island_count > 0:
        contains_cpg_island = True
    in_gene_islands.append((cpg_island_count, contains_cpg_island))


In [ ]:
# in_gene = []
# methy_upstream = []
# methy_downstream = []
# upstream = 1000
# downstream = 3000
# for index, row in dataF.iterrows():
#     in_gene.append(signal_data.where(((row.start <= signal_data.start) & (signal_data.start <= row.end)) | ((row.start <= signal_data.end) & (signal_data.end <= row.end))).mean().fillna(0)[mid])
#     methy_upstream.append(signal_data.where(((row.start - upstream<= signal_data.start) & (signal_data.start <= row.end)) | ((row.start - upstream <= signal_data.end) & (signal_data.end <= row.end))).mean().fillna(0)[mid])
#     methy_downstream.append(signal_data.where(((row.start <= signal_data.start) & (signal_data.start <= row.end + downstream)) | ((row.start <= signal_data.end) & (signal_data.end <= row.end + downstream))).mean().fillna(0)[mid])


In [10]:
pickle.dump( in_gene, open( "in_gene_tumor_buckets.p", "wb" ) )

pickle.dump( methy_upstream, open( "methy_upstream_tumor_buckets.p", "wb" ) )

pickle.dump( methy_downstream, open( "methy_downstream_tumor_buckets.p", "wb" ) )

In [32]:
pickle.dump( in_gene_islands, open( "in_gene_cpg_islands.p", "wb" ) )

In [ ]:
os.getcwd()

In [8]:
in_gene = pickle.load( open( "in_gene.p", "rb" ) )
methy_upstream = pickle.load( open( "methy_upstream.p", "rb" ) )
methy_downstream = pickle.load( open( "methy_downstream.p", "rb" ) )

In [37]:
in_gene = pickle.load( open( "in_gene_tumor_buckets.p", "rb" ) )
in_gene_islands = pickle.load( open( "in_gene_cpg_islands.p", "rb" ) )
methy_upstream = pickle.load( open( "methy_upstream_tumor_buckets.p", "rb" ) )
methy_downstream = pickle.load( open( "methy_downstream_tumor_buckets.p", "rb" ) )

In [38]:
col_names = []
for i in range(20):
    col_names.append("in_gene {}".format(i))
in_gene_df = pd.DataFrame(in_gene, columns = col_names)

col_names = []
for i in range(10):
    col_names.append("upstream {}".format(i))
upstream_df = pd.DataFrame(methy_upstream, columns = col_names)

col_names = []
for i in range(10):
    col_names.append("downstream {}".format(i))
downstream_df = pd.DataFrame(methy_downstream, columns = col_names)

In [36]:
cpg_islands_in_gene = pd.DataFrame(in_gene_islands,columns = ["cpg_count", "contains_cpg_islands"])
cpg_islands_in_gene

,cpg_count,contains_cpg_islands
0,3,True
1,0,False
2,0,False
3,1,True
4,1,True
...,...,...
28584,0,False
28585,0,False
28586,0,False
28587,0,False


In [54]:
cpg_islands_in_gene.cpg_count.value_counts()

0     19640
1      5815
2      1637
3       621
4       295
5       178
6        93
7        80
8        49
11       26
9        25
10       24
13       14
12       12
15       11
18        8
37        7
25        7
16        6
29        6
19        5
17        5
24        4
21        4
14        4
20        3
59        3
30        2
23        1
33        1
27        1
36        1
22        1
Name: cpg_count, dtype: int64

In [39]:
X = pd.concat([in_gene_df,upstream_df,downstream_df,cpg_islands_in_gene], axis=1)
y = pd.Series(dataF["lung___tumor"])

In [19]:
X = pd.DataFrame({'at_gene': in_gene, 'upstream_1000': methy_upstream, 'downstream_3000': methy_downstream}) 
y = pd.Series(dataF["lung___tumor"])

X ["expr"] = y
X = X.dropna()
y = X ["expr"]
X = X[["at_gene", "upstream_1000", "downstream_3000"]]

In [40]:
X

,in_gene 0,in_gene 1,in_gene 2,in_gene 3,in_gene 4,in_gene 5,in_gene 6,in_gene 7,in_gene 8,in_gene 9,...,downstream 2,downstream 3,downstream 4,downstream 5,downstream 6,downstream 7,downstream 8,downstream 9,cpg_count,contains_cpg_islands
0,0.001413,0.000705,0.000001,0.002982,0.001681,0.000237,0.001888,0.000000,0.000891,0.00014,...,0.000980,0.001062,0.000539,0.000718,0.0,0.001323,0.000517,0.000109,3,True
1,0.000530,0.000022,0.000000,0.000469,0.000000,0.000083,0.000340,0.000000,0.000154,0.00000,...,0.000080,0.000189,0.000077,0.000000,0.0,0.000341,0.000199,0.000068,0,False
2,0.000530,0.000022,0.000000,0.000469,0.000000,0.000083,0.000340,0.000000,0.000154,0.00000,...,0.000080,0.000189,0.000077,0.000000,0.0,0.000341,0.000199,0.000068,0,False
3,0.000320,0.000264,0.000000,0.000843,0.000039,0.000000,0.000655,0.000000,0.000000,0.00000,...,0.000124,0.000329,0.000018,0.000072,0.0,0.000501,0.000239,0.000000,1,True
4,0.000225,0.000191,0.000000,0.000031,0.000247,0.000003,0.000159,0.000000,0.000405,0.00000,...,0.000144,0.000177,0.000220,0.000152,0.0,0.000352,0.000151,0.000000,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28584,0.000000,0.000523,0.000035,0.000000,0.000000,0.000000,0.000036,0.000039,0.000000,0.00000,...,0.000000,0.000083,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,False
28585,0.000000,0.000162,0.000000,0.000000,0.000104,0.000000,0.000030,0.000247,0.000000,0.00000,...,0.000072,0.000172,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,False
28586,0.000000,0.000162,0.000000,0.000000,0.000104,0.000000,0.000030,0.000247,0.000000,0.00000,...,0.000072,0.000172,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,False
28587,0.000000,0.000162,0.000000,0.000000,0.000104,0.000000,0.000030,0.000247,0.000000,0.00000,...,0.000072,0.000172,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,False


In [41]:
from sklearn.model_selection import train_test_split

In [48]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)
n_timesteps, n_features, n_outputs = X_train.shape[0], X_train.shape[1], y_train.shape[0]

In [49]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.core import Reshape

In [50]:
verbose, epochs, batch_size = 0, 100, 85

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=(n_features, 1)))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(600, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(250, activation='relu'))
# model.add(Dense(125, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 33, 64)            704       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 704)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 600)               423000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               150250    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                

In [51]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
y_train = np.array(y_train)
# X_train = np.reshape(X_train,(1, X_train.shape[0], X_train.shape[1]))

In [52]:
X_train.shape

(22871, 42, 1)

In [53]:

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
accuracy

0.13098984956741333

In [100]:
accuracy

0.13483735918998718

In [ ]:
sns.pairplot(X)

In [25]:
np.count_nonzero(y) 

1542

In [26]:

# fake_data = np.random.rand(50000,4)
# fake_df = pd.DataFrame({'signal_in_exp': fake_data[:, 0], 'upstream 1000': fake_data[:, 1], 'downstream 3000': fake_data[:, 2], 'expression': fake_data[:, 3]})
model = LinearRegression().fit(X_train, y_train)
print(model.score(X_test, y_test))


0.0623584886969597


In [32]:
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [33]:
svr_poly = SVR(kernel='poly',gamma ='auto')
svr_poly.fit(X_train, y_train)
pred = svr_poly.predict(X_test)
r2_score(y_test, pred)
# svr_poly.score(X_test, y_test)

-0.21593370258836675

In [30]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

-0.0033732316767927095